<a href="https://colab.research.google.com/github/matsunagalab/mdzen/blob/main/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MDZen: AI-Powered Molecular Dynamics Setup

**Run MD simulations with intelligent structure analysis and interactive forms.**

## How It Works

| Step | Description |
|------|-------------|
| **Setup** | Install dependencies (~2-4 min) |
| **Step 1a** | Describe what you want to simulate (free text) |
| **Step 1b** | **Fetch PDB/UniProt info** and analyze structure (chains, ligands, biological unit) |
| **Step 1c** | Answer clarification questions (chain selection, ligand handling, etc.) |
| **Step 1d** | Review & confirm the SimulationBrief (graphical) |
| **Step 2** | Run complete MD workflow |
| **Step 3** | Visualize trajectory |
| **Step 4** | Download results |

---

## Key Features

- **Smart Chain Selection**: Automatically detects if multiple chains are crystallographic copies vs. biological oligomers using UniProt subunit info
- **Ligand Handling**: Identifies bound ligands and asks whether to keep or remove them
- **Graphical Config**: Visual bars and cards for all simulation parameters
- **No Coding Required**: Just fill in forms and click run

## Quick Start

1. Add **API key** in the Setup cell
2. Run **Setup** cell and wait for completion
3. **Describe your simulation** in Step 1a (e.g., "Setup MD for PDB 1AKE")
4. Run Steps 1b-1d to analyze structure and configure
5. Click through Steps 2-4 to run and visualize

---
## Setup: Install Dependencies

**First time only** - This installs AmberTools, OpenMM, and other required packages.

⏱️ Takes ~2-4 minutes. You can continue reading while it runs.

In [ ]:
#@title ▶️ Run Setup (click to expand code)
import sys
import os
import time

IN_COLAB = 'google.colab' in sys.modules
PY_VERSION = f"{sys.version_info.major}.{sys.version_info.minor}"

#==============================================================================
# API Key Configuration (edit here if Colab Secrets doesn't work)
#==============================================================================
# Uncomment ONE of these lines and paste your API key:
# os.environ['ANTHROPIC_API_KEY'] = 'sk-ant-...'
# os.environ['OPENAI_API_KEY'] = 'sk-...'
# os.environ['GOOGLE_API_KEY'] = '...'
#==============================================================================

# Detect and set API keys
detected_provider = None

# 1. Try loading from .env file (for local development)
def load_dotenv():
    """Load environment variables from .env file"""
    for env_path in ['./.env', '../.env', '/content/.env', '/content/mdzen/.env']:
        try:
            with open(env_path) as f:
                for line in f:
                    line = line.strip()
                    if line and not line.startswith('#') and '=' in line:
                        key, value = line.split('=', 1)
                        os.environ[key.strip()] = value.strip().strip('"').strip("'")
                print(f"✓ Loaded .env from {env_path}")
                return True
        except FileNotFoundError:
            continue
    return False

load_dotenv()

# 2. Try Colab secrets (if available)
if IN_COLAB:
    try:
        from google.colab import userdata
        for key_name in ['ANTHROPIC_API_KEY', 'OPENAI_API_KEY', 'GOOGLE_API_KEY']:
            try:
                key_value = userdata.get(key_name)
                if key_value:
                    os.environ[key_name] = key_value
            except:
                pass
    except:
        pass

# 3. Check which API key is available
for key_name, provider in [('ANTHROPIC_API_KEY', 'anthropic'), ('OPENAI_API_KEY', 'openai'), ('GOOGLE_API_KEY', 'google')]:
    if os.environ.get(key_name):
        detected_provider = provider
        print(f"✓ {key_name} ({provider})")
        break

if not detected_provider:
    print("⚠️ No API key found!")
    print("   → Edit this cell and uncomment the API key line above, OR")
    print("   → Add to Colab Secrets (if using native Colab)")

if IN_COLAB:
    start_time = time.time()
    print(f"\n🐍 Python {PY_VERSION}")
    
    # Ensure we're in a valid directory first
    os.chdir('/content')
    
    # Install Miniforge (includes mamba for fast solving)
    print("📦 Installing Miniforge...")
    !curl -fsSL https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh -o /tmp/miniforge.sh
    !bash /tmp/miniforge.sh -b -p /usr/local -u 2>&1 | tail -1
    os.environ["PATH"] = f"/usr/local/bin:{os.environ['PATH']}"
    print(f"✓ Miniforge ({time.time() - start_time:.0f}s)")

    # Use mamba for fast package installation
    print("⚗️ Installing AmberTools + dependencies...")
    !mamba create -n mdzen python=3.11 ambertools=23 openmm rdkit pdbfixer -y -q 2>&1 | grep -E "(done|Total)" | tail -3
    print(f"✓ Conda packages ({time.time() - start_time:.0f}s)")

    print("📥 Cloning repository...")
    !rm -rf /content/mdzen
    !git clone -q https://github.com/matsunagalab/mdzen.git /content/mdzen
    os.chdir('/content/mdzen')

    # Install uv and Python packages (uv with explicit python path)
    print("📦 Installing Python packages (using uv)...")
    !pip install -q uv
    PYTHON_PATH = sys.executable
    !uv pip install --python {PYTHON_PATH} -q \
        "litellm>=1.60.0,<1.80.0" \
        anthropic \
        google-genai \
        google-adk \
        "fastmcp>=2.0.0" \
        "mcp[cli]" \
        gradio \
        py3Dmol \
        nest_asyncio \
        mdtraj \
        gemmi \
        pdb2pqr \
        propka \
        dimorphite-dl
    print(f"✓ Python packages ({time.time() - start_time:.0f}s)")

    # Set environment variables
    os.environ["AMBERHOME"] = "/usr/local/envs/mdzen"
    os.environ["MDZEN_CONDA_ENV"] = "mdzen"
    os.environ["PATH"] = f"/usr/local/envs/mdzen/bin:{os.environ['PATH']}"
    
    # Add paths
    sys.path.insert(0, '/content/mdzen/src')
    sys.path.insert(0, '/content/mdzen')

    # Quick verification
    try:
        import litellm, gradio, fastmcp
        from google.adk.runners import Runner
        print("✓ Core packages verified")
    except ImportError as e:
        print(f"⚠️ Import check: {e}")

    # Start MCP servers with Streamable HTTP transport (recommended)
    print("🚀 Starting MCP servers (Streamable HTTP mode)...")
    import subprocess
    
    MCP_SERVERS = [
        ("research_server.py", 8001),
        ("structure_server.py", 8002),
        ("genesis_server.py", 8003),
        ("solvation_server.py", 8004),
        ("amber_server.py", 8005),
        ("md_simulation_server.py", 8006),
    ]
    
    mcp_server_procs = []
    for server_file, port in MCP_SERVERS:
        proc = subprocess.Popen(
            [sys.executable, f"/content/mdzen/servers/{server_file}", "--http", "--port", str(port)],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
        )
        mcp_server_procs.append((server_file, port, proc))
        print(f"   ✓ {server_file} on port {port} (/mcp)")
    
    time.sleep(3)  # Wait for servers to initialize
    print(f"✓ MCP servers started ({time.time() - start_time:.0f}s)")

    print(f"\n✅ Setup complete! ({(time.time() - start_time)/60:.1f} min)")
else:
    sys.path.insert(0, './src')
    sys.path.insert(0, '.')
    load_dotenv()
    print("Local environment")

---
## Step 1: Describe Your Simulation

Tell the AI what you want to simulate in plain language. The AI will ask clarifying questions to help set up the perfect simulation.

In [ ]:
#@title 🧬 Step 1a: Describe Your Simulation { display-mode: "form" }
#@markdown ### What do you want to simulate?
user_request = "I want to run MD simulation of PDB 1AKE (adenylate kinase) in water at 300K for 1 ns" #@param {type:"string"}

#@markdown ---
#@markdown ### Examples (copy one if you like):
#@markdown - `Setup MD for PDB 1AKE in explicit water, 1 ns at 300K`
#@markdown - `Simulate lysozyme (1LYZ) with TIP3P water model`
#@markdown - `Run equilibrium simulation of ubiquitin (1UBQ) at 310K`
#@markdown - `Setup protein-ligand complex from 3HTB for drug binding study`

import sys
import json
import random
import string
from pathlib import Path

# Initialize session
IN_COLAB = 'google.colab' in sys.modules
if 'mdzen_state' not in dir():
    mdzen_state = {
        "session_id": None, 
        "session_dir": None, 
        "user_request": None,
        "clarification_questions": None,
        "user_answers": None,
        "simulation_brief": None, 
        "workflow_outputs": {}
    }

def init_session():
    job_id = ''.join(random.choices(string.ascii_lowercase + string.digits, k=8))
    base_dir = Path("/content/mdzen/outputs") if IN_COLAB else Path("./outputs")
    session_dir = base_dir / f"job_{job_id}"
    session_dir.mkdir(parents=True, exist_ok=True)
    mdzen_state["session_id"] = f"job_{job_id}"
    mdzen_state["session_dir"] = str(session_dir)
    return session_dir

if mdzen_state["session_dir"] is None:
    init_session()

# Save user request
mdzen_state["user_request"] = user_request.strip()

print("=" * 60)
print("  ✅ Request Received!")
print("=" * 60)
print(f"  📝 \"{user_request}\"")
print("=" * 60)
print(f"\n📁 Session: {mdzen_state['session_id']}")
print("\n👉 Run the next cell to get AI clarification questions")

In [ ]:
#@title 🤖 Step 1b: Structure Analysis & Clarification (ADK Runner) { display-mode: "form" }
#@markdown This cell uses the ADK Runner with MCP tools to analyze structure and generate questions.
#@markdown 
#@markdown **Run this cell**, then answer the questions in the next cell.

import re
import json
from pathlib import Path
from IPython.display import display, HTML

if 'mdzen_state' not in dir() or not mdzen_state.get("user_request"):
    print("❌ Error: Please run Step 1a first")
else:
    user_request = mdzen_state["user_request"]
    session_dir = mdzen_state["session_dir"]
    
    print("🤖 Starting clarification agent (ADK Runner + MCP Streamable HTTP)...")
    print("-" * 60)
    
    # Import shared agent (same as main.py!)
    from mdzen.agents.clarification_agent import create_clarification_agent
    from mdzen.tools.mcp_setup import close_toolsets
    
    from google.adk.runners import Runner
    from google.adk.sessions import InMemorySessionService
    from google.genai import types
    
    # Create agent with HTTP transport (Streamable HTTP /mcp endpoint)
    agent, mcp_tools = create_clarification_agent(transport="http")
    
    # Create runner (same pattern as main.py!)
    session_service = InMemorySessionService()
    runner = Runner(
        app_name="mdzen",
        agent=agent,
        session_service=session_service,
    )
    
    # Run the agent
    async def run_clarification():
        session = await session_service.create_session(
            app_name="mdzen",
            user_id="colab_user",
            state={"session_dir": session_dir},
        )
        
        message = types.Content(
            role="user",
            parts=[types.Part(text=user_request)],
        )
        
        final_response = None
        async for event in runner.run_async(
            user_id="colab_user",
            session_id=session.id,
            new_message=message,
        ):
            if event.is_final_response() and event.content:
                final_response = event.content.parts[0].text if event.content.parts else None
        
        # Get simulation brief from session state
        updated_session = await session_service.get_session(
            app_name="mdzen",
            user_id="colab_user",
            session_id=session.id,
        )
        
        return final_response, updated_session.state
    
    try:
        # Colab allows direct await
        final_response, session_state = await run_clarification()
        
        # Close MCP connections
        await close_toolsets(mcp_tools)
        
        if session_state.get("simulation_brief"):
            brief = session_state["simulation_brief"]
            # Parse if JSON string
            if isinstance(brief, str):
                try:
                    brief = json.loads(brief)
                except json.JSONDecodeError:
                    # Not valid JSON - might be a clarification question
                    pass
            
            # Check if we got a valid dict brief or a text response
            if isinstance(brief, dict):
                mdzen_state["simulation_brief"] = brief
                
                print("✅ Clarification complete!")
                print("-" * 60)
                print(f"\n📋 Generated SimulationBrief:")
                for key, val in brief.items():
                    if val is not None:
                        print(f"   • {key}: {val}")
                
                print()
                print("=" * 60)
                print("👉 Run Step 1d to review and modify the configuration")
                print("   (Step 1c is optional - use if you need to answer questions manually)")
            else:
                # Agent returned text (clarification questions)
                print("\n🤖 Agent needs more information:")
                print("-" * 60)
                print(brief)
                print()
                print("=" * 60)
                print("👉 Please answer the questions above, then re-run this cell")
                print("   or proceed to Step 1c for manual configuration")
        else:
            # Display agent's response for manual interaction
            print("\n🤖 Agent response:")
            print(final_response or "No response")
            print()
            print("👉 Check the agent's response and proceed to Step 1c if needed")
            
    except Exception as e:
        import traceback
        print(f"❌ Error: {e}")
        print(traceback.format_exc())
        
        # Cleanup
        try:
            await close_toolsets(mcp_tools)
        except:
            pass

In [ ]:
#@title 📝 Step 1c: Answer Questions (Optional - for manual mode) { display-mode: "form" }
#@markdown ### Your Answers
#@markdown **This step is optional if Step 1b generated a SimulationBrief directly.**
#@markdown 
#@markdown Enter the option number (0, 1, 2, or 3) for each question.
#@markdown Check the output of Step 1b to see the options.

answer_1 = 0 #@param {type:"integer"}
answer_2 = 0 #@param {type:"integer"}
answer_3 = 0 #@param {type:"integer"}
answer_4 = 0 #@param {type:"integer"}
answer_5 = 0 #@param {type:"integer"}

#@markdown ---
#@markdown ### Additional notes (optional)
additional_notes = "" #@param {type:"string"}

import json

# Check if we already have a simulation brief from Step 1b
if 'mdzen_state' in dir() and mdzen_state.get("simulation_brief"):
    print("✅ SimulationBrief already generated by Step 1b!")
    print("   You can skip this cell and go directly to Step 1d.")
    print()
    brief = mdzen_state["simulation_brief"]
    print("   Current brief summary:")
    print(f"   • PDB ID: {brief.get('pdb_id')}")
    if brief.get('select_chains'):
        print(f"   • Chains: {brief.get('select_chains')}")
    print(f"   • Temperature: {brief.get('temperature', 300)} K")
    print(f"   • Duration: {brief.get('simulation_time_ns', 1.0)} ns")
    print()
    print("👉 Run Step 1d to review and modify the configuration")
    
elif 'mdzen_state' not in dir() or not mdzen_state.get("clarification_questions"):
    print("❌ Error: Please run Step 1b first")
    print("   If Step 1b failed, you may need to check your MCP server setup.")
else:
    from mdzen.tools.clarification_utils import (
        StructureAnalysis,
        ClarificationQuestion,
        generate_brief_from_answers,
    )
    
    result = mdzen_state["clarification_questions"]
    questions_data = result.get("questions", [])
    answers = [answer_1, answer_2, answer_3, answer_4, answer_5]
    
    # Reconstruct ClarificationQuestion objects
    questions = [
        ClarificationQuestion(
            id=q["id"],
            question=q["question"],
            options=q["options"],
            default=q.get("default", 0),
            context=q.get("context", ""),
        )
        for q in questions_data
    ]
    
    # Display selected answers
    print("📋 Your answers:")
    print("-" * 50)
    for i, q in enumerate(questions):
        if i < len(answers):
            ans_idx = answers[i]
            if ans_idx < len(q.options):
                print(f"   {q.id}: {q.options[ans_idx]}")
    
    if additional_notes.strip():
        print(f"   notes: {additional_notes}")
    
    # Store answers
    mdzen_state["user_answers"] = answers[:len(questions)]
    
    print()
    print("🤖 Generating SimulationBrief...")
    print("-" * 50)
    
    # Reconstruct StructureAnalysis from stored data
    analysis_data = mdzen_state.get("structure_analysis", {})
    analysis = StructureAnalysis.from_dict(analysis_data)
    
    # Generate brief using shared utility
    brief = generate_brief_from_answers(
        analysis=analysis,
        questions=questions,
        answers=answers[:len(questions)],
        user_request=mdzen_state.get("user_request", ""),
    )
    
    # Handle additional notes
    if additional_notes.strip():
        # Parse any chain specifications from notes
        import re
        chain_match = re.search(r'chain[s]?\s*[:\s]?\s*([A-Za-z,\s]+)', additional_notes.lower())
        if chain_match:
            chains = [c.strip().upper() for c in chain_match.group(1).split(",")]
            brief["select_chains"] = chains
    
    mdzen_state["simulation_brief"] = brief
    
    print("✅ SimulationBrief generated from structure analysis!")
    print()
    print("   Summary:")
    print(f"   • PDB ID: {brief.get('pdb_id')}")
    if brief.get('select_chains'):
        print(f"   • Chains: {brief.get('select_chains')}")
    print(f"   • Temperature: {brief.get('temperature')} K")
    print(f"   • Duration: {brief.get('simulation_time_ns')} ns")
    print(f"   • Water: {brief.get('water_model')}")
    print(f"   • Force field: {brief.get('force_field')}")
    print()
    print("👉 Run the next cell to review and confirm")

In [ ]:
#@title ✅ Step 1d: Review & Confirm SimulationBrief { run: "auto", display-mode: "form" }
#@markdown ### 📦 Structure
pdb_id = "1AKE" #@param {type:"string"}
select_chains = "" #@param {type:"string"}
#@markdown > Leave chains empty for all, or specify "A" or "A,B"

#@markdown ---
#@markdown ### 🌡️ Simulation Conditions
temperature = 300 #@param {type:"slider", min:250, max:400, step:10}
pressure_bar = 1.0 #@param {type:"number"}
simulation_time_ns = 1.0 #@param {type:"slider", min:0.1, max:10, step:0.1}
ensemble = "NPT" #@param ["NPT", "NVT"]

#@markdown ---
#@markdown ### 💧 Solvation
water_model = "tip3p" #@param ["tip3p", "opc", "spce", "tip4pew"]
box_padding = 12.0 #@param {type:"slider", min:8, max:20, step:1}
salt_concentration = 0.15 #@param {type:"slider", min:0, max:0.3, step:0.01}
cubic_box = True #@param {type:"boolean"}

#@markdown ---
#@markdown ### ⚙️ Force Field & pH
force_field = "ff19SB" #@param ["ff19SB", "ff14SB", "ff99SB"]
ph = 7.4 #@param {type:"slider", min:5.0, max:9.0, step:0.1}

from IPython.display import display, HTML

def make_bar(value, min_val, max_val, width=20):
    """Create a visual bar representation"""
    pct = max(0, min(1, (value - min_val) / (max_val - min_val)))
    filled = int(pct * width)
    bar = "█" * filled + "░" * (width - filled)
    return bar

def display_brief_graphical(brief, analysis_data=None):
    """Display SimulationBrief with graphical elements and structure info"""
    
    # Temperature bar (250-400K)
    temp_bar = make_bar(brief['temperature'], 250, 400)
    
    # Time bar (0.1-10 ns)
    time_bar = make_bar(brief['simulation_time_ns'], 0.1, 10)
    
    # Box padding bar (8-20 Å)
    box_bar = make_bar(brief['box_padding'], 8, 20)
    
    # Salt bar (0-0.3 M)
    salt_bar = make_bar(brief['salt_concentration'], 0, 0.3)
    
    # pH bar (5-9)
    ph_bar = make_bar(brief['ph'], 5, 9)
    
    # Structure info from analysis
    structure_info = ""
    if analysis_data:
        title = analysis_data.get("title", "")[:50]
        subunit = analysis_data.get("subunit_composition", "")
        protein_chains = analysis_data.get("protein_chain_ids", [])
        ligands = analysis_data.get("ligands", [])
        
        chains_str = ", ".join(protein_chains) if protein_chains else "N/A"
        selected = brief.get('select_chains')
        if selected:
            if isinstance(selected, list):
                selected_str = ", ".join(selected)
            else:
                selected_str = selected
        else:
            selected_str = "All"
        
        ligand_str = ", ".join([l.get("comp_id", "?") for l in ligands]) if ligands else "None"
        
        structure_info = f"""
        <div class="brief-section" style="grid-column: span 2;">
            <span class="brief-icon">🧬</span>
            <span class="brief-label">STRUCTURE INFO</span><br>
            <div style="display: grid; grid-template-columns: 1fr 1fr 1fr; gap: 10px; margin-top: 8px;">
                <div>
                    <span style="color:#888; font-size:11px;">Title</span><br>
                    <span style="font-size:12px;">{title}...</span>
                </div>
                <div>
                    <span style="color:#888; font-size:11px;">Biological Unit</span><br>
                    <span style="font-size:14px; color:#fbbf24;">{subunit or 'Unknown'}</span>
                </div>
                <div>
                    <span style="color:#888; font-size:11px;">Available Chains</span><br>
                    <span style="font-size:14px;">{chains_str}</span>
                </div>
            </div>
            <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 10px; margin-top: 10px;">
                <div>
                    <span style="color:#888; font-size:11px;">Selected Chains</span><br>
                    <span style="font-size:16px; font-weight:bold; color:#4ade80;">{selected_str}</span>
                </div>
                <div>
                    <span style="color:#888; font-size:11px;">Ligands</span><br>
                    <span style="font-size:14px;">{ligand_str}</span>
                </div>
            </div>
        </div>
        """
    
    html = f"""
    <style>
        .brief-container {{ font-family: monospace; background: #1e1e1e; color: #fff; padding: 20px; border-radius: 10px; }}
        .brief-title {{ font-size: 18px; font-weight: bold; margin-bottom: 15px; color: #4CAF50; }}
        .brief-section {{ margin: 15px 0; padding: 10px; background: #2d2d2d; border-radius: 5px; }}
        .brief-label {{ color: #888; font-size: 12px; }}
        .brief-value {{ font-size: 16px; font-weight: bold; color: #fff; }}
        .brief-bar {{ color: #4CAF50; letter-spacing: 2px; }}
        .brief-bar-salt {{ color: #2196F3; letter-spacing: 2px; }}
        .brief-bar-temp {{ color: #FF5722; letter-spacing: 2px; }}
        .brief-bar-ph {{ color: #9C27B0; letter-spacing: 2px; }}
        .brief-grid {{ display: grid; grid-template-columns: 1fr 1fr; gap: 10px; }}
        .brief-icon {{ font-size: 20px; margin-right: 8px; }}
    </style>
    <div class="brief-container">
        <div class="brief-title">✅ SimulationBrief Confirmed</div>
        
        <div class="brief-section">
            <span class="brief-icon">📦</span>
            <span class="brief-label">PDB ID</span><br>
            <span class="brief-value">{brief['pdb_id']}</span>
            {f" <span style='color:#4ade80'>(chains: {brief['select_chains'] if isinstance(brief.get('select_chains'), str) else ', '.join(brief.get('select_chains', []))})</span>" if brief.get('select_chains') else " <span style='color:#888'>(all chains)</span>"}
        </div>
        
        {structure_info}
        
        <div class="brief-grid">
            <div class="brief-section">
                <span class="brief-icon">🌡️</span>
                <span class="brief-label">TEMPERATURE</span><br>
                <span class="brief-value">{brief['temperature']} K</span><br>
                <span class="brief-bar-temp">{temp_bar}</span>
                <span style="color:#666; font-size:10px"> 250━━━━━━━━━━400K</span>
            </div>
            <div class="brief-section">
                <span class="brief-icon">⏱️</span>
                <span class="brief-label">DURATION</span><br>
                <span class="brief-value">{brief['simulation_time_ns']} ns</span><br>
                <span class="brief-bar">{time_bar}</span>
                <span style="color:#666; font-size:10px"> 0.1━━━━━━━━━━10ns</span>
            </div>
        </div>
        
        <div class="brief-grid">
            <div class="brief-section">
                <span class="brief-icon">💧</span>
                <span class="brief-label">WATER MODEL</span><br>
                <span class="brief-value">{brief['water_model'].upper()}</span>
            </div>
            <div class="brief-section">
                <span class="brief-icon">📐</span>
                <span class="brief-label">BOX PADDING</span><br>
                <span class="brief-value">{brief['box_padding']} Å</span><br>
                <span class="brief-bar">{box_bar}</span>
                <span style="color:#666; font-size:10px"> 8━━━━━━━━━━20Å</span>
            </div>
        </div>
        
        <div class="brief-grid">
            <div class="brief-section">
                <span class="brief-icon">🧂</span>
                <span class="brief-label">SALT CONCENTRATION</span><br>
                <span class="brief-value">{brief['salt_concentration']} M</span><br>
                <span class="brief-bar-salt">{salt_bar}</span>
                <span style="color:#666; font-size:10px"> 0━━━━━━━━━━0.3M</span>
            </div>
            <div class="brief-section">
                <span class="brief-icon">🧪</span>
                <span class="brief-label">pH</span><br>
                <span class="brief-value">{brief['ph']}</span><br>
                <span class="brief-bar-ph">{ph_bar}</span>
                <span style="color:#666; font-size:10px"> 5━━━━━━━━━━9</span>
            </div>
        </div>
        
        <div class="brief-grid">
            <div class="brief-section">
                <span class="brief-icon">⚙️</span>
                <span class="brief-label">FORCE FIELD</span><br>
                <span class="brief-value">{brief['force_field']}</span>
            </div>
            <div class="brief-section">
                <span class="brief-icon">📊</span>
                <span class="brief-label">ENSEMBLE</span><br>
                <span class="brief-value">{brief['ensemble']}</span>
                <span style="color:#888">({brief['pressure_bar']} bar)</span>
            </div>
        </div>
    </div>
    """
    display(HTML(html))

# Check if we have AI-generated brief and use its values as defaults
if 'mdzen_state' in dir() and mdzen_state.get("simulation_brief"):
    ai_brief = mdzen_state["simulation_brief"]
    print("🤖 AI-generated values (modify form above if needed):")
    for key, val in ai_brief.items():
        if val is not None and key not in ["structure_file", "title", "subunit_composition", "include_types"]:
            print(f"   {key}: {val}")
    print()

# Build final brief from form values
final_brief = {
    "pdb_id": pdb_id.strip().upper(),
    "temperature": temperature,
    "simulation_time_ns": simulation_time_ns,
    "water_model": water_model,
    "force_field": force_field,
    "box_padding": box_padding,
    "salt_concentration": salt_concentration,
    "select_chains": select_chains.strip() if select_chains.strip() else None,
    "pressure_bar": pressure_bar,
    "ensemble": ensemble,
    "cubic_box": cubic_box,
    "ph": ph,
}

# Validate
if len(final_brief["pdb_id"]) != 4:
    print("❌ Error: PDB ID must be exactly 4 characters")
else:
    mdzen_state["simulation_brief"] = final_brief
    
    # Get structure analysis if available
    analysis_data = mdzen_state.get("structure_analysis") if 'mdzen_state' in dir() else None
    
    display_brief_graphical(final_brief, analysis_data)
    print(f"\n📁 Output: {mdzen_state.get('session_dir', 'Not set')}")
    print("\n👉 Run Step 2 to start the MD workflow")

---
## Step 2: Run MD Workflow

This will execute all 4 steps automatically:
1. **prepare_complex** - Download structure and prepare proteins/ligands
2. **solvate** - Add water box and ions  
3. **build_topology** - Generate Amber topology files
4. **run_simulation** - Run MD with OpenMM

Click ▶️ to start. Progress will be shown below.

In [ ]:
#@title ⚙️ Step 2: Run Complete Workflow (ADK Runner) { display-mode: "form" }
#@markdown ### Run Options
run_simulation_step = True #@param {type:"boolean"}
#@markdown > Uncheck to skip the MD simulation (for testing setup only)

import sys
import json
import traceback
from pathlib import Path
import time

# Check prerequisites
if 'mdzen_state' not in dir() or not mdzen_state.get("simulation_brief"):
    print("❌ Error: Please run Step 1 first to configure your simulation")
else:
    brief = mdzen_state["simulation_brief"]
    session_dir = Path(mdzen_state["session_dir"])
    
    print("=" * 60)
    print(f"  🚀 Starting MD Workflow for {brief.get('pdb_id', 'Unknown')}")
    print("  📡 Using ADK Runner + MCP Streamable HTTP Transport")
    print("=" * 60)
    
    # Import shared agent (same as main.py!)
    from mdzen.agents.setup_agent import create_setup_agent
    from mdzen.tools.mcp_setup import close_toolsets
    
    from google.adk.runners import Runner
    from google.adk.sessions import InMemorySessionService
    from google.genai import types
    
    # Create agent with HTTP transport (Streamable HTTP /mcp endpoint)
    agent, mcp_tools = create_setup_agent(transport="http")
    
    # Create runner (same pattern as main.py!)
    session_service = InMemorySessionService()
    runner = Runner(
        app_name="mdzen",
        agent=agent,
        session_service=session_service,
    )
    
    async def run_setup():
        # Initialize session with simulation brief
        initial_state = {
            "session_dir": str(session_dir),
            "simulation_brief": json.dumps(brief) if isinstance(brief, dict) else brief,
            "completed_steps": json.dumps([]),
            "outputs": json.dumps({}),
        }
        
        session = await session_service.create_session(
            app_name="mdzen",
            user_id="colab_user",
            state=initial_state,
        )
        
        # Build the setup request
        steps_to_run = ["prepare_complex", "solvate", "build_topology"]
        if run_simulation_step:
            steps_to_run.append("run_simulation")
        
        request = f"""Execute the MD setup workflow with the following SimulationBrief:

{json.dumps(brief, indent=2)}

Please run these steps in order: {', '.join(steps_to_run)}

Work in the directory: {session_dir}
"""
        
        message = types.Content(
            role="user",
            parts=[types.Part(text=request)],
        )
        
        print("\n🤖 Setup agent is running...")
        print("-" * 60)
        
        final_response = None
        async for event in runner.run_async(
            user_id="colab_user",
            session_id=session.id,
            new_message=message,
        ):
            # Print progress updates from the agent
            if event.content and event.content.parts:
                text = event.content.parts[0].text if hasattr(event.content.parts[0], 'text') else None
                if text and not event.is_final_response():
                    # Print intermediate responses (progress updates)
                    if any(kw in text.lower() for kw in ['step', 'complete', 'running', 'preparing', 'building']):
                        print(f"   {text[:200]}...")
                
            if event.is_final_response() and event.content:
                final_response = event.content.parts[0].text if event.content.parts else None
        
        # Get outputs from session state
        updated_session = await session_service.get_session(
            app_name="mdzen",
            user_id="colab_user",
            session_id=session.id,
        )
        
        return final_response, updated_session.state
    
    try:
        start_time = time.time()
        
        # Colab allows direct await
        final_response, session_state = await run_setup()
        
        # Close MCP connections
        await close_toolsets(mcp_tools)
        
        elapsed = time.time() - start_time
        
        # Extract outputs from session state
        outputs = session_state.get("outputs", {})
        if isinstance(outputs, str):
            try:
                outputs = json.loads(outputs)
            except:
                outputs = {}
        
        completed = session_state.get("completed_steps", [])
        if isinstance(completed, str):
            try:
                completed = json.loads(completed)
            except:
                completed = []
        
        # Store outputs for visualization
        mdzen_state["workflow_outputs"] = outputs
        
        print()
        print("=" * 60)
        print("  🎉 Workflow Complete!")
        print("=" * 60)
        print(f"  ⏱️ Time: {elapsed/60:.1f} min")
        print(f"  ✅ Steps completed: {', '.join(completed) if completed else 'None'}")
        print(f"  📁 Output: {session_dir}")
        
        # List key output files
        if outputs:
            print()
            print("  📦 Generated files:")
            for key, path in outputs.items():
                if path:
                    print(f"     • {key}: {Path(path).name if isinstance(path, str) else path}")
        
        print()
        if final_response:
            print("  📝 Agent summary:")
            # Print first 500 chars of response
            summary = final_response[:500] + "..." if len(final_response) > 500 else final_response
            for line in summary.split('\n'):
                print(f"     {line}")
        
        print()
        print("  👉 Run the next cell to visualize the trajectory")
        
    except Exception as e:
        print()
        print("=" * 60)
        print(f"  ❌ Error: {e}")
        print("=" * 60)
        print(traceback.format_exc())
        
        # Cleanup
        try:
            await close_toolsets(mcp_tools)
        except:
            pass

---
## Step 3: Visualize Results

View the trajectory animation with py3Dmol.

In [ ]:
#@title 🎬 Step 3: Visualize Trajectory { display-mode: "form" }
#@markdown ### Visualization Options
max_frames = 20 #@param {type:"slider", min:5, max:50, step:5}
style = "cartoon" #@param ["cartoon", "stick", "sphere", "line"]

import py3Dmol
import numpy as np
import tempfile
from pathlib import Path

if 'mdzen_state' not in dir() or not mdzen_state.get("workflow_outputs"):
    print("❌ Error: Please run the workflow first (Step 2)")
elif 'trajectory' not in mdzen_state["workflow_outputs"]:
    print("❌ Error: No trajectory found. Make sure 'Run simulation' was checked in Step 2")
else:
    print("📊 Loading trajectory...")
    
    import mdtraj as md
    traj = md.load(
        mdzen_state["workflow_outputs"]['trajectory'], 
        top=mdzen_state["workflow_outputs"]['parm7']
    )
    
    # Select protein atoms only
    protein = traj.atom_slice(traj.topology.select('protein'))
    
    # Sample frames
    if protein.n_frames > max_frames:
        indices = np.linspace(0, protein.n_frames-1, max_frames, dtype=int)
        frames = protein[indices]
    else:
        frames = protein
    
    # Create multi-model PDB
    with tempfile.NamedTemporaryFile(suffix='.pdb', delete=False, mode='w') as tmp:
        for i in range(frames.n_frames):
            ftmp = f"{tmp.name}.f{i}.pdb"
            frames[i].save_pdb(ftmp, force_overwrite=True)
            tmp.write(f"MODEL {i+1}\n")
            with open(ftmp) as f:
                tmp.write(''.join(l for l in f if not l.startswith(('MODEL','ENDMDL')) and l.strip()))
            tmp.write("ENDMDL\n")
            Path(ftmp).unlink()
        tmp_path = tmp.name
    
    # Create viewer
    view = py3Dmol.view(width=800, height=500)
    with open(tmp_path) as f:
        view.addModelsAsFrames(f.read(), 'pdb')
    
    # Apply style
    if style == "cartoon":
        view.setStyle({'cartoon': {'color': 'spectrum'}})
    elif style == "stick":
        view.setStyle({'stick': {}})
    elif style == "sphere":
        view.setStyle({'sphere': {'radius': 0.5}})
    else:
        view.setStyle({'line': {}})
    
    view.zoomTo()
    view.animate({'loop': 'forward', 'reps': 0, 'interval': 100})
    
    Path(tmp_path).unlink()
    
    print(f"✅ Loaded {frames.n_frames} frames ({traj.time[-1]:.1f} ps total)")
    print(f"   Style: {style}")
    print()
    print("👉 Run the next cell to download all files")
    
    view.show()

---
## Step 4: Download Results

Download all generated files as a ZIP archive.

In [ ]:
#@title 📥 Step 4: Download Results { display-mode: "form" }

import sys
from pathlib import Path

if 'mdzen_state' not in dir() or not mdzen_state.get("session_dir"):
    print("❌ Error: Please run the workflow first")
else:
    session_dir = Path(mdzen_state["session_dir"])
    
    if not session_dir.exists():
        print("❌ Error: Session directory not found")
    else:
        print("=" * 50)
        print(f"  📂 {session_dir.name}")
        print("=" * 50)
        
        files = sorted(session_dir.rglob('*'))
        total_size = 0
        
        for f in files:
            if f.is_file():
                size = f.stat().st_size
                total_size += size
                size_str = f"{size/1024:.1f} KB" if size > 1024 else f"{size} B"
                print(f"  {f.relative_to(session_dir):<40} {size_str:>10}")
        
        print("=" * 50)
        print(f"  Total: {total_size/1024/1024:.2f} MB")
        print("=" * 50)
        
        if 'google.colab' in sys.modules:
            from google.colab import files
            import shutil
            
            zip_path = f"/content/{session_dir.name}.zip"
            shutil.make_archive(zip_path.replace('.zip', ''), 'zip', session_dir)
            
            print(f"\n⬇️ Downloading {session_dir.name}.zip...")
            files.download(zip_path)
        else:
            print(f"\n📁 Files are in: {session_dir}")

---

## What's Next?

🎉 **Congratulations!** You've completed an MD simulation using MDZen.

### Continue Learning
- **Longer simulations**: Increase `simulation_time_ns` in Step 1
- **Different proteins**: Try other PDB IDs like 1LYZ (lysozyme), 1UBQ (ubiquitin)
- **Analysis**: Load trajectory in MDTraj for RMSD, RMSF, hydrogen bonds

### Resources
- [MDZen GitHub](https://github.com/matsunagalab/mdzen)
- [OpenMM Documentation](http://openmm.org/documentation.html)
- [MDTraj Documentation](https://mdtraj.org/)

In [ ]:
# Check if we're in a local environment (not Colab)
import sys
print(f"Python: {sys.version}")
print(f"In Colab: {'google.colab' in sys.modules}")

# Add paths for local development
sys.path.insert(0, './src')
sys.path.insert(0, '.')

# Check if API key is available
import os
from pathlib import Path

# Try to load .env
def load_dotenv():
    for env_path in ['./.env', '../.env']:
        try:
            with open(env_path) as f:
                for line in f:
                    line = line.strip()
                    if line and not line.startswith('#') and '=' in line:
                        key, value = line.split('=', 1)
                        os.environ[key.strip()] = value.strip().strip('"').strip("'")
                print(f"✓ Loaded .env from {env_path}")
                return True
        except FileNotFoundError:
            continue
    return False

load_dotenv()

# Check API keys
api_key = os.environ.get('ANTHROPIC_API_KEY', '')
if api_key:
    print(f"✓ ANTHROPIC_API_KEY: {api_key[:10]}...")
else:
    print("✗ No ANTHROPIC_API_KEY found")


In [ ]:
# Start MCP servers with Streamable HTTP transport
import subprocess
import sys
import time
import os

# Get the correct path
base_path = os.getcwd()
print(f"Working directory: {base_path}")

MCP_SERVERS = [
    ("research_server.py", 8001),
    ("structure_server.py", 8002),
    ("genesis_server.py", 8003),
    ("solvation_server.py", 8004),
    ("amber_server.py", 8005),
    ("md_simulation_server.py", 8006),
]

mcp_server_procs = []
for server_file, port in MCP_SERVERS:
    server_path = f"{base_path}/servers/{server_file}"
    proc = subprocess.Popen(
        [sys.executable, server_path, "--http", "--port", str(port)],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
    )
    mcp_server_procs.append((server_file, port, proc))
    print(f"   ✓ {server_file} on port {port} (/mcp)")

time.sleep(3)  # Wait for servers to initialize
print(f"\n✓ MCP servers started")


In [ ]:
# Step 1a: Initialize session
import sys
import json
import random
import string
from pathlib import Path

# Initialize session state
IN_COLAB = 'google.colab' in sys.modules
mdzen_state = {
    "session_id": None, 
    "session_dir": None, 
    "user_request": None,
    "clarification_questions": None,
    "user_answers": None,
    "simulation_brief": None, 
    "workflow_outputs": {}
}

def init_session():
    job_id = ''.join(random.choices(string.ascii_lowercase + string.digits, k=8))
    base_dir = Path("/content/mdzen/outputs") if IN_COLAB else Path("./outputs")
    session_dir = base_dir / f"job_{job_id}"
    session_dir.mkdir(parents=True, exist_ok=True)
    mdzen_state["session_id"] = f"job_{job_id}"
    mdzen_state["session_dir"] = str(session_dir)
    return session_dir

init_session()

# Set user request
user_request = "I want to run MD simulation of PDB 1AKE (adenylate kinase) in water at 300K for 1 ns"
mdzen_state["user_request"] = user_request.strip()

print("=" * 60)
print("  ✅ Request Received!")
print("=" * 60)
print(f"  📝 \"{user_request}\"")
print("=" * 60)
print(f"\n📁 Session: {mdzen_state['session_id']}")
print(f"📂 Directory: {mdzen_state['session_dir']}")


In [ ]:
# Step 1b: Structure Analysis & Clarification (ADK Runner)
import re
import json
from pathlib import Path

user_request = mdzen_state["user_request"]
session_dir = mdzen_state["session_dir"]

print("🤖 Starting clarification agent (ADK Runner + MCP Streamable HTTP)...")
print("-" * 60)

# Import shared agent (same as main.py!)
from mdzen.agents.clarification_agent import create_clarification_agent
from mdzen.tools.mcp_setup import close_toolsets

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Create agent with HTTP transport (Streamable HTTP /mcp endpoint)
agent, mcp_tools = create_clarification_agent(transport="http")

# Create runner (same pattern as main.py!)
session_service = InMemorySessionService()
runner = Runner(
    app_name="mdzen",
    agent=agent,
    session_service=session_service,
)

# Run the agent
async def run_clarification():
    session = await session_service.create_session(
        app_name="mdzen",
        user_id="colab_user",
        state={"session_dir": session_dir},
    )
    
    message = types.Content(
        role="user",
        parts=[types.Part(text=user_request)],
    )
    
    final_response = None
    async for event in runner.run_async(
        user_id="colab_user",
        session_id=session.id,
        new_message=message,
    ):
        if event.is_final_response() and event.content:
            final_response = event.content.parts[0].text if event.content.parts else None
    
    # Get simulation brief from session state
    updated_session = await session_service.get_session(
        app_name="mdzen",
        user_id="colab_user",
        session_id=session.id,
    )
    
    return final_response, updated_session.state

try:
    # Run the async function
    final_response, session_state = await run_clarification()
    
    # Close MCP connections
    await close_toolsets(mcp_tools)
    
    if session_state.get("simulation_brief"):
        brief = session_state["simulation_brief"]
        # Parse if JSON string
        if isinstance(brief, str):
            try:
                brief = json.loads(brief)
            except:
                pass
        
        if isinstance(brief, dict):
            mdzen_state["simulation_brief"] = brief
            
            print("✅ Clarification complete!")
            print("-" * 60)
            print(f"\n📋 Generated SimulationBrief:")
            for key, val in brief.items():
                if val is not None:
                    print(f"   • {key}: {val}")
            
            print()
            print("=" * 60)
            print("👉 Ready for Step 2")
        else:
            # Agent returned clarification questions (not a JSON brief)
            print("\n🤖 Agent needs more information:")
            print(brief[:500] if len(brief) > 500 else brief)
    else:
        # Display agent's response for manual interaction
        print("\n🤖 Agent response:")
        print(final_response or "No response")
        
except Exception as e:
    import traceback
    print(f"❌ Error: {e}")
    print(traceback.format_exc())
    
    # Cleanup
    try:
        await close_toolsets(mcp_tools)
    except:
        pass

In [ ]:
# Reload the modules to pick up changes
import importlib
import sys

# Remove cached modules
modules_to_remove = [k for k in sys.modules.keys() if k.startswith('mdzen')]
for mod in modules_to_remove:
    del sys.modules[mod]

print(f"Removed {len(modules_to_remove)} cached mdzen modules")

# Re-import
from mdzen.agents.clarification_agent import create_clarification_agent
import inspect
print(f"create_clarification_agent signature: {inspect.signature(create_clarification_agent)}")


In [ ]:
# Check what's in the content directory
import os
content_file = "/content/mdzen/src/mdzen/agents/clarification_agent.py"
print(f"Reading: {content_file}")

with open(content_file, 'r') as f:
    content = f.read()

# Check if transport parameter exists
if "transport:" in content:
    print("✓ transport parameter found")
else:
    print("✗ transport parameter NOT found - need to update")

# Show function definition line
for i, line in enumerate(content.split('\n'), 1):
    if 'def create_clarification_agent' in line:
        print(f"Line {i}: {line}")
        # Show next few lines
        lines = content.split('\n')
        for j in range(i, min(i+5, len(lines))):
            print(f"Line {j+1}: {lines[j]}")
        break


In [ ]:
# Update clarification_agent.py with transport parameter
clarification_agent_code = '''"""Phase 1: Clarification Agent for MDZen.

This agent handles user interaction to gather MD simulation requirements
and generates a structured SimulationBrief.
"""

from typing import Literal

from google.adk.agents import LlmAgent
from google.adk.models.lite_llm import LiteLlm
from google.adk.tools.function_tool import FunctionTool
from google.adk.tools.mcp_tool import McpToolset

from mdzen.config import get_litellm_model
from mdzen.prompts import get_clarification_instruction
from mdzen.tools.mcp_setup import get_clarification_tools, get_clarification_tools_sse
from mdzen.tools.custom_tools import generate_simulation_brief, get_session_dir


def create_clarification_agent(
    transport: Literal["stdio", "sse", "http"] = "stdio",
    sse_host: str = "localhost",
) -> tuple[LlmAgent, list[McpToolset]]:
    """Create the Phase 1 clarification agent.

    This agent:
    1. Gets session_dir via get_session_dir tool
    2. Uses download_structure/inspect_molecules to analyze structures
    3. Asks clarification questions based on inspection
    4. Generates SimulationBrief via generate_simulation_brief tool
    5. Saves result to session.state["simulation_brief"] via output_key

    Args:
        transport: MCP transport mode:
            - "stdio": subprocess-based (default, for CLI)
            - "sse" or "http": HTTP-based using Streamable HTTP /mcp endpoint (for Colab)
        sse_host: Hostname for HTTP servers (only used when transport="sse" or "http")

    Returns:
        Tuple of (LlmAgent, list of McpToolset instances to close after use)
    """
    # Get MCP tools for structure inspection based on transport mode
    if transport in ("sse", "http"):
        mcp_tools = get_clarification_tools_sse(host=sse_host)
    else:
        mcp_tools = get_clarification_tools()

    # Create FunctionTools for session management and brief generation
    get_session_dir_tool = FunctionTool(get_session_dir)
    generate_brief_tool = FunctionTool(generate_simulation_brief)

    # Combine all tools
    all_tools = mcp_tools + [get_session_dir_tool, generate_brief_tool]

    agent = LlmAgent(
        model=LiteLlm(model=get_litellm_model("clarification")),
        name="clarification_agent",
        description="Gathers MD simulation requirements and generates SimulationBrief",
        instruction=get_clarification_instruction(),
        tools=all_tools,
        output_key="simulation_brief",  # Saves to session.state["simulation_brief"]
    )

    return agent, mcp_tools
'''

with open("/content/mdzen/src/mdzen/agents/clarification_agent.py", 'w') as f:
    f.write(clarification_agent_code)

print("✓ Updated clarification_agent.py")


In [ ]:
import subprocess
import os

os.chdir("/content/mdzen")
result = subprocess.run(["git", "pull", "origin", "main"], capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print(result.stderr)


In [ ]:
# Kill old MCP server processes and restart with new code
import subprocess
import sys
import os
import time

# Kill existing MCP servers
for server_file, port, proc in mcp_server_procs:
    try:
        proc.terminate()
        proc.wait(timeout=2)
    except:
        proc.kill()

print("✓ Stopped old MCP servers")

# Clear module cache
modules_to_remove = [k for k in sys.modules.keys() if k.startswith('mdzen')]
for mod in modules_to_remove:
    del sys.modules[mod]
print(f"✓ Cleared {len(modules_to_remove)} cached modules")

# Restart MCP servers
base_path = "/content/mdzen"
MCP_SERVERS = [
    ("research_server.py", 8001),
    ("structure_server.py", 8002),
    ("genesis_server.py", 8003),
    ("solvation_server.py", 8004),
    ("amber_server.py", 8005),
    ("md_simulation_server.py", 8006),
]

mcp_server_procs = []
for server_file, port in MCP_SERVERS:
    server_path = f"{base_path}/servers/{server_file}"
    proc = subprocess.Popen(
        [sys.executable, server_path, "--http", "--port", str(port)],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
    )
    mcp_server_procs.append((server_file, port, proc))
    print(f"   ✓ {server_file} on port {port}")

time.sleep(3)
print("\n✓ MCP servers restarted with new code")

# Verify transport parameter exists
from mdzen.agents.clarification_agent import create_clarification_agent
import inspect
print(f"\ncreate_clarification_agent signature: {inspect.signature(create_clarification_agent)}")


In [ ]:
# Step 1b: Structure Analysis & Clarification (ADK Runner)
import re
import json
from pathlib import Path

user_request = mdzen_state["user_request"]
session_dir = mdzen_state["session_dir"]

print("🤖 Starting clarification agent (ADK Runner + MCP Streamable HTTP)...")
print("-" * 60)

# Import shared agent
from mdzen.agents.clarification_agent import create_clarification_agent
from mdzen.tools.mcp_setup import close_toolsets

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Create agent with HTTP transport (Streamable HTTP /mcp endpoint)
agent, mcp_tools = create_clarification_agent(transport="http")
print("✓ Agent created with HTTP transport")

# Create runner
session_service = InMemorySessionService()
runner = Runner(
    app_name="mdzen",
    agent=agent,
    session_service=session_service,
)

# Run the agent
async def run_clarification():
    session = await session_service.create_session(
        app_name="mdzen",
        user_id="colab_user",
        state={"session_dir": session_dir},
    )
    
    message = types.Content(
        role="user",
        parts=[types.Part(text=user_request)],
    )
    
    final_response = None
    async for event in runner.run_async(
        user_id="colab_user",
        session_id=session.id,
        new_message=message,
    ):
        if event.is_final_response() and event.content:
            final_response = event.content.parts[0].text if event.content.parts else None
    
    # Get simulation brief from session state
    updated_session = await session_service.get_session(
        app_name="mdzen",
        user_id="colab_user",
        session_id=session.id,
    )
    
    return final_response, updated_session.state

try:
    print("🔄 Running clarification agent...")
    final_response, session_state = await run_clarification()
    
    # Close MCP connections
    await close_toolsets(mcp_tools)
    
    if session_state.get("simulation_brief"):
        brief = session_state["simulation_brief"]
        if isinstance(brief, str):
            try:
                brief = json.loads(brief)
            except:
                pass
        
        if isinstance(brief, dict):
            mdzen_state["simulation_brief"] = brief
            
            print("✅ Clarification complete!")
            print("-" * 60)
            print(f"\n📋 Generated SimulationBrief:")
            for key, val in brief.items():
                if val is not None:
                    print(f"   • {key}: {val}")
        else:
            # Agent returned clarification questions (not a JSON brief)
            print("\n🤖 Agent needs more information:")
            print(brief[:500] if len(brief) > 500 else brief)
    else:
        print("\n🤖 Agent response:")
        print(final_response or "No response")
        
except Exception as e:
    import traceback
    print(f"❌ Error: {e}")
    print(traceback.format_exc())
    try:
        await close_toolsets(mcp_tools)
    except:
        pass

In [ ]:
# Check if MCP servers are running and listening
import subprocess

# Check running processes
result = subprocess.run(["ps", "aux"], capture_output=True, text=True)
python_procs = [line for line in result.stdout.split('\n') if 'server.py' in line and 'python' in line]
print("Running server processes:")
for proc in python_procs:
    print(f"  {proc[:100]}...")

print()

# Check if ports are listening
result = subprocess.run(["ss", "-tlnp"], capture_output=True, text=True)
print("Listening ports:")
for line in result.stdout.split('\n'):
    if any(str(port) in line for port in [8001, 8002, 8003, 8004, 8005, 8006]):
        print(f"  {line}")

# Try to connect to one of the servers
import socket
for port in [8001, 8002]:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)
    result = sock.connect_ex(('localhost', port))
    if result == 0:
        print(f"✓ Port {port} is open")
    else:
        print(f"✗ Port {port} is NOT open (error: {result})")
    sock.close()


In [ ]:
# Start one server with visible output to see what's happening
import subprocess
import sys
import time

base_path = "/content/mdzen"
server_path = f"{base_path}/servers/research_server.py"

print(f"Starting: python {server_path} --http --port 8001")
print("-" * 60)

# Start with captured output
proc = subprocess.Popen(
    [sys.executable, server_path, "--http", "--port", "8001"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,
)

# Wait a bit and check
time.sleep(3)

# Check if process is still running
if proc.poll() is None:
    print("✓ Server process is running")
else:
    print(f"✗ Server exited with code: {proc.returncode}")
    stdout, stderr = proc.communicate()
    if stdout:
        print("STDOUT:", stdout[:1000])
    if stderr:
        print("STDERR:", stderr[:1000])


In [ ]:
# Check if port 8001 is now listening
import socket
import time

time.sleep(2)  # Give more time for server to bind

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.settimeout(2)
result = sock.connect_ex(('localhost', 8001))
if result == 0:
    print("✓ Port 8001 is open")
else:
    print(f"✗ Port 8001 is NOT open (error: {result})")
sock.close()

# Try HTTP request to the /mcp endpoint
import urllib.request
try:
    req = urllib.request.Request('http://localhost:8001/mcp', method='GET')
    with urllib.request.urlopen(req, timeout=5) as response:
        print(f"HTTP response: {response.status}")
except Exception as e:
    print(f"HTTP request result: {e}")


In [ ]:
# Check server process output
import subprocess
import sys

# Kill any existing and start fresh with full logging
subprocess.run(["pkill", "-f", "research_server"], capture_output=True)

import time
time.sleep(1)

base_path = "/content/mdzen"
server_path = f"{base_path}/servers/research_server.py"

# Run server and wait for output
proc = subprocess.Popen(
    [sys.executable, server_path, "--http", "--port", "8001"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
)

# Wait and read output
time.sleep(5)

# Read any available output
import select
import os

# Make stdout non-blocking
os.set_blocking(proc.stdout.fileno(), False)

output = ""
try:
    output = proc.stdout.read() or ""
except:
    pass

print(f"Process running: {proc.poll() is None}")
print(f"Output so far:\n{output[:2000] if output else '(no output yet)'}")

# Check stderr separately
if proc.poll() is not None:
    print(f"Exit code: {proc.returncode}")


In [ ]:
# Check FastMCP version and run() signature
import fastmcp
import inspect

print(f"FastMCP version: {fastmcp.__version__}")

# Check FastMCP.run signature
from fastmcp import FastMCP
mcp = FastMCP("test")
sig = inspect.signature(mcp.run)
print(f"\nFastMCP.run() signature: {sig}")

# Check available parameters
print("\nrun() parameters:")
for name, param in sig.parameters.items():
    print(f"  {name}: {param.annotation} = {param.default}")


In [ ]:
# Check what transports are available and how to configure them
from fastmcp import FastMCP

# Check if there are transport-specific settings
import fastmcp.server
print(dir(fastmcp.server))

# Try to find how to set port
try:
    from fastmcp.server import serve
    sig = inspect.signature(serve)
    print(f"\nserve() signature: {sig}")
except:
    pass

# Check Transport type
try:
    from fastmcp.server.transports import Transport
    print(f"\nTransport types available:")
    for name in dir(fastmcp.server):
        if 'transport' in name.lower():
            print(f"  {name}")
except Exception as e:
    print(f"Error: {e}")


In [ ]:
# Pull latest changes and restart servers
import subprocess
import sys
import os
import time

os.chdir("/content/mdzen")

# Pull changes
result = subprocess.run(["git", "pull", "origin", "main"], capture_output=True, text=True)
print(result.stdout)

# Kill any existing server processes
subprocess.run(["pkill", "-f", "_server.py"], capture_output=True)
time.sleep(1)

# Clear module cache
modules_to_remove = [k for k in sys.modules.keys() if k.startswith('mdzen')]
for mod in modules_to_remove:
    del sys.modules[mod]
print(f"✓ Cleared {len(modules_to_remove)} cached modules")

# Restart servers
base_path = "/content/mdzen"
MCP_SERVERS = [
    ("research_server.py", 8001),
    ("structure_server.py", 8002),
    ("genesis_server.py", 8003),
    ("solvation_server.py", 8004),
    ("amber_server.py", 8005),
    ("md_simulation_server.py", 8006),
]

mcp_server_procs = []
for server_file, port in MCP_SERVERS:
    server_path = f"{base_path}/servers/{server_file}"
    proc = subprocess.Popen(
        [sys.executable, server_path, "--http", "--port", str(port)],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
    )
    mcp_server_procs.append((server_file, port, proc))

time.sleep(3)

# Check if port 8001 is open
import socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.settimeout(2)
result = sock.connect_ex(('localhost', 8001))
if result == 0:
    print("✓ Port 8001 is open - server is running!")
else:
    print(f"✗ Port 8001 connection failed (error: {result})")
sock.close()


In [ ]:
# Check server startup with visible output
import subprocess
import sys
import time

base_path = "/content/mdzen"
server_path = f"{base_path}/servers/research_server.py"

# Kill any existing
subprocess.run(["pkill", "-f", "research_server"], capture_output=True)
time.sleep(1)

print(f"Starting: {server_path} --http --port 8001")
print("-" * 60)

# Start with captured output
proc = subprocess.Popen(
    [sys.executable, server_path, "--http", "--port", "8001"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)

# Wait longer for uvicorn to start
time.sleep(5)

# Check if still running
if proc.poll() is None:
    print("✓ Process still running")
    
    # Check port
    import socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)
    result = sock.connect_ex(('localhost', 8001))
    if result == 0:
        print("✓ Port 8001 is listening!")
    else:
        print(f"✗ Port 8001 not open yet (error: {result})")
    sock.close()
else:
    print(f"✗ Process exited with code: {proc.returncode}")
    
# Read any output
import os
os.set_blocking(proc.stdout.fileno(), False)
try:
    output = proc.stdout.read() or ""
    if output:
        print("\nServer output:")
        print(output[:2000])
except:
    pass


In [ ]:
# Check FastMCP run method more carefully
from fastmcp import FastMCP
import inspect

mcp = FastMCP("test")

# Get the source of run method
print("FastMCP.run source:")
print(inspect.getsource(mcp.run)[:2000])


In [ ]:
# Check run_async method
from fastmcp import FastMCP
import inspect

mcp = FastMCP("test")

# Get the source of run_async method
print("FastMCP.run_async source:")
source = inspect.getsource(mcp.run_async)
print(source[:3000])


In [ ]:
# Check run_http_async method
from fastmcp import FastMCP
import inspect

mcp = FastMCP("test")

# Get the signature
sig = inspect.signature(mcp.run_http_async)
print(f"run_http_async signature: {sig}")

# Get the source
print("\nrun_http_async source:")
source = inspect.getsource(mcp.run_http_async)
print(source[:2500])


In [ ]:
# Try setting config via FastMCP settings instead
from fastmcp import FastMCP

# Check if there's a settings/config way
mcp = FastMCP("test")
print(f"FastMCP version: {fastmcp.__version__}")
print(f"Settings attributes: {[a for a in dir(mcp) if 'setting' in a.lower() or 'config' in a.lower()]}")

# Check _deprecated_settings
if hasattr(mcp, '_deprecated_settings'):
    print(f"\n_deprecated_settings: {mcp._deprecated_settings}")
    print(f"host: {mcp._deprecated_settings.host}")
    print(f"port: {mcp._deprecated_settings.port}")


In [ ]:
# Try setting host/port via the deprecated_settings before run
from fastmcp import FastMCP
import fastmcp

print(f"FastMCP version: {fastmcp.__version__}")

# Create server and configure settings
mcp = FastMCP("test")
mcp._deprecated_settings.host = "0.0.0.0"
mcp._deprecated_settings.port = 8099

print(f"Configured host: {mcp._deprecated_settings.host}")
print(f"Configured port: {mcp._deprecated_settings.port}")

# Try run with just transport
# This should work and use the settings we just set
print("\nTrying mcp.run(transport='http') - this should use the settings...")
# Don't actually run it, just verify the approach
print("(Not actually running, just testing approach)")

# Also check if there's a direct way to configure in constructor
print("\nChecking FastMCP constructor signature:")
import inspect
sig = inspect.signature(FastMCP.__init__)
print(f"FastMCP.__init__{sig}")


In [ ]:
# Pull latest and restart servers
import subprocess
import sys
import os
import time
import socket

os.chdir("/content/mdzen")

# Pull changes
result = subprocess.run(["git", "pull", "origin", "main"], capture_output=True, text=True)
print(result.stdout)

# Kill any existing server processes
subprocess.run(["pkill", "-f", "_server.py"], capture_output=True)
time.sleep(1)

# Clear module cache
modules_to_remove = [k for k in sys.modules.keys() if k.startswith('mdzen')]
for mod in modules_to_remove:
    del sys.modules[mod]

# Start research_server and test
base_path = "/content/mdzen"
server_path = f"{base_path}/servers/research_server.py"

print(f"\nStarting research_server on port 8001...")
proc = subprocess.Popen(
    [sys.executable, server_path, "--http", "--port", "8001"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)

time.sleep(5)

# Check if running and port open
if proc.poll() is None:
    print("✓ Process is running")
    
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)
    result = sock.connect_ex(('localhost', 8001))
    if result == 0:
        print("✓ Port 8001 is listening!")
    else:
        print(f"✗ Port 8001 not open (error: {result})")
    sock.close()
else:
    print(f"✗ Process exited with code: {proc.returncode}")
    os.set_blocking(proc.stdout.fileno(), False)
    try:
        output = proc.stdout.read() or ""
        print(f"Output: {output[:1500]}")
    except:
        pass


In [ ]:
# Check FastMCP version in Colab
import fastmcp
print(f"FastMCP version: {fastmcp.__version__}")

from fastmcp import FastMCP
mcp = FastMCP("test")
print(f"Available attributes: {[a for a in dir(mcp) if not a.startswith('__')]}")


In [ ]:
# Run a minimal test to see what's happening
import subprocess
import sys

test_code = '''
import sys
sys.path.insert(0, '/content/mdzen/src')
from fastmcp import FastMCP
import fastmcp
print(f"FastMCP version: {fastmcp.__version__}")
mcp = FastMCP("test")
print(f"Has _deprecated_settings: {hasattr(mcp, '_deprecated_settings')}")
if hasattr(mcp, '_deprecated_settings'):
    print(f"Settings: {mcp._deprecated_settings}")
else:
    print(f"Settings: {mcp.settings if hasattr(mcp, 'settings') else 'No settings'}")
'''

result = subprocess.run([sys.executable, "-c", test_code], capture_output=True, text=True)
print("STDOUT:", result.stdout)
if result.stderr:
    print("STDERR:", result.stderr[:500])


In [ ]:
# Check the research_server.py directly
import subprocess
import sys

# Run a test to check what's happening in the server file
result = subprocess.run(
    [sys.executable, "-c", '''
import sys
sys.path.insert(0, "/content/mdzen/src")

# Import what research_server imports
from fastmcp import FastMCP
import fastmcp
print(f"FastMCP version: {fastmcp.__version__}")

# Create mcp same way as research_server
mcp = FastMCP("Research Server")
print(f"mcp type: {type(mcp)}")
print(f"Has _deprecated_settings: {hasattr(mcp, '_deprecated_settings')}")

# Try setting like in the server
try:
    mcp._deprecated_settings.host = "0.0.0.0"
    mcp._deprecated_settings.port = 8001
    print("✓ Settings applied successfully")
except AttributeError as e:
    print(f"✗ Error: {e}")
'''],
    capture_output=True,
    text=True
)
print("STDOUT:", result.stdout)
if result.stderr:
    print("STDERR:", result.stderr[:1000])


In [ ]:
# Let's see exactly what's happening in the server file
import subprocess
import sys

result = subprocess.run(
    [sys.executable, "/content/mdzen/servers/research_server.py", "--http", "--port", "8001"],
    capture_output=True,
    text=True,
    timeout=10
)
print("Exit code:", result.returncode)
print("STDOUT:", result.stdout[:500] if result.stdout else "")
print("STDERR:", result.stderr[:1500] if result.stderr else "")


In [ ]:
# Check what's different in the server environment
import subprocess
import sys

# Check if there's a local fastmcp module or something shadowing
result = subprocess.run(
    [sys.executable, "-c", '''
import sys
sys.path.insert(0, "/content/mdzen/src")
sys.path.insert(0, "/content/mdzen/servers")
sys.path.insert(0, "/content/mdzen")

# Debug imports
import fastmcp
print(f"fastmcp location: {fastmcp.__file__}")
print(f"fastmcp version: {fastmcp.__version__}")

from fastmcp import FastMCP
print(f"FastMCP class: {FastMCP}")
print(f"FastMCP module: {FastMCP.__module__}")

# Check all fastmcp-related modules
for name, mod in sorted(sys.modules.items()):
    if 'fastmcp' in name.lower() or 'mcp' in name.lower():
        if hasattr(mod, '__file__'):
            print(f"  {name}: {mod.__file__}")
'''],
    capture_output=True,
    text=True
)
print("STDOUT:", result.stdout)
if result.stderr:
    print("STDERR:", result.stderr[:500])


In [ ]:
# Check the first few lines of research_server.py to see how it imports FastMCP
with open("/content/mdzen/servers/research_server.py", "r") as f:
    lines = f.readlines()
    for i, line in enumerate(lines[:50], 1):
        if 'import' in line.lower() or 'fastmcp' in line.lower():
            print(f"{i}: {line.rstrip()}")


In [ ]:
# Pull and test
import subprocess
import sys
import os
import time
import socket

os.chdir("/content/mdzen")

# Pull changes
result = subprocess.run(["git", "pull", "origin", "main"], capture_output=True, text=True)
print(result.stdout)

# Kill any existing server processes
subprocess.run(["pkill", "-f", "_server.py"], capture_output=True)
time.sleep(1)

# Start research_server and test
base_path = "/content/mdzen"
server_path = f"{base_path}/servers/research_server.py"

print(f"\nStarting research_server on port 8001...")
proc = subprocess.Popen(
    [sys.executable, server_path, "--http", "--port", "8001"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)

time.sleep(5)

# Check if running and port open
if proc.poll() is None:
    print("✓ Process is running")
    
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)
    result = sock.connect_ex(('localhost', 8001))
    if result == 0:
        print("✓ Port 8001 is listening!")
    else:
        print(f"✗ Port 8001 not open (error: {result})")
    sock.close()
else:
    print(f"✗ Process exited with code: {proc.returncode}")
    os.set_blocking(proc.stdout.fileno(), False)
    try:
        output = proc.stdout.read() or ""
        print(f"Output:\n{output[:2000]}")
    except:
        pass


In [ ]:
# Start all MCP servers
import subprocess
import sys
import os
import time
import socket

# Kill any existing
subprocess.run(["pkill", "-f", "_server.py"], capture_output=True)
time.sleep(1)

# Clear module cache
modules_to_remove = [k for k in sys.modules.keys() if k.startswith('mdzen')]
for mod in modules_to_remove:
    del sys.modules[mod]

base_path = "/content/mdzen"
MCP_SERVERS = [
    ("research_server.py", 8001),
    ("structure_server.py", 8002),
    ("genesis_server.py", 8003),
    ("solvation_server.py", 8004),
    ("amber_server.py", 8005),
    ("md_simulation_server.py", 8006),
]

mcp_server_procs = []
for server_file, port in MCP_SERVERS:
    server_path = f"{base_path}/servers/{server_file}"
    proc = subprocess.Popen(
        [sys.executable, server_path, "--http", "--port", str(port)],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
    )
    mcp_server_procs.append((server_file, port, proc))

time.sleep(5)

# Check all ports
for server_file, port, proc in mcp_server_procs:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)
    result = sock.connect_ex(('localhost', port))
    status = "✓" if result == 0 else "✗"
    print(f"{status} {server_file} on port {port}")
    sock.close()


In [ ]:
# Check what's happening with the servers
import subprocess
import sys
import os
import time

base_path = "/content/mdzen"

# Start one server with output visible
server_path = f"{base_path}/servers/research_server.py"

proc = subprocess.Popen(
    [sys.executable, server_path, "--http", "--port", "8001"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)

# Wait and check
time.sleep(8)

if proc.poll() is None:
    print("✓ Server is running")
    
    # Check port
    import socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)
    result = sock.connect_ex(('localhost', 8001))
    print(f"Port check: {'✓ open' if result == 0 else '✗ not open'}")
    sock.close()
else:
    print(f"✗ Server exited with code: {proc.returncode}")
    
# Read output
os.set_blocking(proc.stdout.fileno(), False)
try:
    output = proc.stdout.read() or ""
    if output:
        print(f"\nOutput:\n{output[:1500]}")
except:
    pass


In [ ]:
# Run server directly and capture all output
import subprocess
import sys

result = subprocess.run(
    [sys.executable, "/content/mdzen/servers/research_server.py", "--http", "--port", "8001"],
    capture_output=True,
    text=True,
    timeout=15
)

print("Exit code:", result.returncode)
print("\nSTDOUT:")
print(result.stdout[-3000:] if len(result.stdout) > 3000 else result.stdout)
print("\nSTDERR:")
print(result.stderr[-2000:] if len(result.stderr) > 2000 else result.stderr)


In [ ]:
# Kill all server processes properly
import subprocess
import time
import os

# Kill by port
for port in [8001, 8002, 8003, 8004, 8005, 8006]:
    subprocess.run(f"fuser -k {port}/tcp 2>/dev/null", shell=True, capture_output=True)

# Also kill by name
subprocess.run(["pkill", "-9", "-f", "_server.py"], capture_output=True)

time.sleep(2)

# Verify ports are free
import socket
for port in [8001, 8002, 8003, 8004, 8005, 8006]:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    result = sock.connect_ex(('localhost', port))
    status = "in use" if result == 0 else "free"
    print(f"Port {port}: {status}")
    sock.close()


In [ ]:
# Start all MCP servers with proper waiting
import subprocess
import sys
import time
import socket

base_path = "/content/mdzen"
MCP_SERVERS = [
    ("research_server.py", 8001),
    ("structure_server.py", 8002),
    ("genesis_server.py", 8003),
    ("solvation_server.py", 8004),
    ("amber_server.py", 8005),
    ("md_simulation_server.py", 8006),
]

mcp_server_procs = []
for server_file, port in MCP_SERVERS:
    server_path = f"{base_path}/servers/{server_file}"
    proc = subprocess.Popen(
        [sys.executable, server_path, "--http", "--port", str(port)],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
    )
    mcp_server_procs.append((server_file, port, proc))
    print(f"Started {server_file}...")
    time.sleep(1)  # Stagger startup

print("\nWaiting for servers to initialize...")
time.sleep(5)

# Check all ports
print("\nServer status:")
all_ok = True
for server_file, port, proc in mcp_server_procs:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)
    result = sock.connect_ex(('localhost', port))
    if result == 0:
        print(f"✓ {server_file} on port {port}")
    else:
        print(f"✗ {server_file} on port {port} (process running: {proc.poll() is None})")
        all_ok = False
    sock.close()

if all_ok:
    print("\n✅ All servers running!")


In [ ]:
# Check structure_server error
import subprocess
import sys

result = subprocess.run(
    [sys.executable, "/content/mdzen/servers/structure_server.py", "--http", "--port", "8002"],
    capture_output=True,
    text=True,
    timeout=10
)

print("Exit code:", result.returncode)
print("\nError output (last 2000 chars):")
print(result.stderr[-2000:] if result.stderr else "(no stderr)")


In [ ]:
# Clear module cache and run clarification agent (only needs research_server)
import sys

# Clear cached mdzen modules
modules_to_remove = [k for k in sys.modules.keys() if k.startswith('mdzen')]
for mod in modules_to_remove:
    del sys.modules[mod]
print(f"Cleared {len(modules_to_remove)} cached modules")

# Now run Step 1b
import json
from pathlib import Path

user_request = mdzen_state["user_request"]
session_dir = mdzen_state["session_dir"]

print(f"\n🤖 Starting clarification agent...")
print(f"   Request: {user_request}")
print(f"   Session: {session_dir}")
print("-" * 60)

from mdzen.agents.clarification_agent import create_clarification_agent
from mdzen.tools.mcp_setup import close_toolsets

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Create agent with HTTP transport
agent, mcp_tools = create_clarification_agent(transport="http")
print("✓ Agent created")

# Create runner
session_service = InMemorySessionService()
runner = Runner(
    app_name="mdzen",
    agent=agent,
    session_service=session_service,
)

async def run_clarification():
    session = await session_service.create_session(
        app_name="mdzen",
        user_id="colab_user",
        state={"session_dir": session_dir},
    )
    
    message = types.Content(
        role="user",
        parts=[types.Part(text=user_request)],
    )
    
    print("🔄 Running agent...")
    final_response = None
    async for event in runner.run_async(
        user_id="colab_user",
        session_id=session.id,
        new_message=message,
    ):
        if event.is_final_response() and event.content:
            final_response = event.content.parts[0].text if event.content.parts else None
    
    updated_session = await session_service.get_session(
        app_name="mdzen",
        user_id="colab_user",
        session_id=session.id,
    )
    
    return final_response, updated_session.state

try:
    final_response, session_state = await run_clarification()
    await close_toolsets(mcp_tools)
    
    if session_state.get("simulation_brief"):
        brief = session_state["simulation_brief"]
        if isinstance(brief, str):
            try:
                brief = json.loads(brief)
            except:
                pass
        
        if isinstance(brief, dict):
            mdzen_state["simulation_brief"] = brief
            
            print("\n✅ Clarification complete!")
            print("-" * 60)
            print("📋 Generated SimulationBrief:")
            for key, val in brief.items():
                if val is not None:
                    print(f"   • {key}: {val}")
        else:
            # Agent returned clarification questions (not a JSON brief)
            print("\n🤖 Agent needs more information:")
            print(brief[:1000] if len(brief) > 1000 else brief)
    else:
        print("\n🤖 Agent response:")
        print(final_response[:1000] if final_response else "No response")
        
except Exception as e:
    import traceback
    print(f"\n❌ Error: {e}")
    traceback.print_exc()
    try:
        await close_toolsets(mcp_tools)
    except:
        pass

In [ ]:
# Kill existing and restart with SSE mode
import subprocess
import sys
import time
import socket

# Kill all
for port in [8001, 8002, 8003, 8004, 8005, 8006]:
    subprocess.run(f"fuser -k {port}/tcp 2>/dev/null", shell=True, capture_output=True)
subprocess.run(["pkill", "-9", "-f", "_server.py"], capture_output=True)
time.sleep(2)

# Start research_server with SSE mode
base_path = "/content/mdzen"
server_path = f"{base_path}/servers/research_server.py"

print("Starting research_server with SSE mode...")
proc = subprocess.Popen(
    [sys.executable, server_path, "--sse", "--port", "8001"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)

time.sleep(5)

# Check port
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.settimeout(2)
result = sock.connect_ex(('localhost', 8001))
if result == 0:
    print("✓ Port 8001 is listening")
else:
    print(f"✗ Port 8001 not open")
sock.close()

# Test SSE endpoint
import urllib.request
try:
    req = urllib.request.Request('http://localhost:8001/sse')
    with urllib.request.urlopen(req, timeout=5) as response:
        print(f"✓ /sse endpoint responded: {response.status}")
except Exception as e:
    print(f"SSE endpoint: {e}")


In [ ]:
# Check what connection types ADK supports
from google.adk.tools.mcp_tool import mcp_session_manager
import inspect

# List all classes in the module
print("Available connection params in ADK:")
for name in dir(mcp_session_manager):
    obj = getattr(mcp_session_manager, name)
    if isinstance(obj, type) and 'Params' in name:
        print(f"  - {name}")
        
# Check SseConnectionParams
from google.adk.tools.mcp_tool.mcp_session_manager import SseConnectionParams
print(f"\nSseConnectionParams fields:")
print(inspect.signature(SseConnectionParams))


In [ ]:
# Check StreamableHTTPConnectionParams signature
from google.adk.tools.mcp_tool.mcp_session_manager import StreamableHTTPConnectionParams
import inspect

print("StreamableHTTPConnectionParams:")
print(inspect.signature(StreamableHTTPConnectionParams))


In [ ]:
# Pull, restart servers, and test
import subprocess
import sys
import os
import time
import socket

os.chdir("/content/mdzen")

# Pull changes
result = subprocess.run(["git", "pull", "origin", "main"], capture_output=True, text=True)
print(result.stdout)

# Kill existing servers
for port in [8001, 8002, 8003, 8004, 8005, 8006]:
    subprocess.run(f"fuser -k {port}/tcp 2>/dev/null", shell=True, capture_output=True)
subprocess.run(["pkill", "-9", "-f", "_server.py"], capture_output=True)
time.sleep(2)

# Clear module cache
modules_to_remove = [k for k in sys.modules.keys() if k.startswith('mdzen')]
for mod in modules_to_remove:
    del sys.modules[mod]
print(f"Cleared {len(modules_to_remove)} cached modules")

# Start research_server with HTTP mode
base_path = "/content/mdzen"
server_path = f"{base_path}/servers/research_server.py"

print("\nStarting research_server with HTTP mode...")
proc = subprocess.Popen(
    [sys.executable, server_path, "--http", "--port", "8001"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)

time.sleep(5)

# Check port
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.settimeout(2)
result = sock.connect_ex(('localhost', 8001))
if result == 0:
    print("✓ research_server on port 8001")
else:
    print(f"✗ Port 8001 not open")
sock.close()


In [ ]:
# Test clarification agent with Streamable HTTP
import json

user_request = mdzen_state["user_request"]
session_dir = mdzen_state["session_dir"]

print(f"🤖 Testing clarification agent with Streamable HTTP...")
print(f"   Request: {user_request[:60]}...")
print("-" * 60)

from mdzen.agents.clarification_agent import create_clarification_agent
from mdzen.tools.mcp_setup import close_toolsets

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Create agent with HTTP transport (now uses StreamableHTTPConnectionParams)
agent, mcp_tools = create_clarification_agent(transport="http")
print("✓ Agent created with StreamableHTTPConnectionParams")

# Create runner
session_service = InMemorySessionService()
runner = Runner(
    app_name="mdzen",
    agent=agent,
    session_service=session_service,
)

async def run_clarification():
    session = await session_service.create_session(
        app_name="mdzen",
        user_id="colab_user",
        state={"session_dir": session_dir},
    )
    
    message = types.Content(
        role="user",
        parts=[types.Part(text=user_request)],
    )
    
    print("🔄 Running agent...")
    final_response = None
    async for event in runner.run_async(
        user_id="colab_user",
        session_id=session.id,
        new_message=message,
    ):
        if event.is_final_response() and event.content:
            final_response = event.content.parts[0].text if event.content.parts else None
    
    updated_session = await session_service.get_session(
        app_name="mdzen",
        user_id="colab_user",
        session_id=session.id,
    )
    
    return final_response, updated_session.state

try:
    final_response, session_state = await run_clarification()
    await close_toolsets(mcp_tools)
    
    if session_state.get("simulation_brief"):
        brief = session_state["simulation_brief"]
        if isinstance(brief, str):
            try:
                brief = json.loads(brief)
            except:
                pass
        
        if isinstance(brief, dict):
            mdzen_state["simulation_brief"] = brief
            
            print("\n✅ SUCCESS! Clarification complete!")
            print("-" * 60)
            print("📋 Generated SimulationBrief:")
            for key, val in brief.items():
                if val is not None:
                    print(f"   • {key}: {val}")
        else:
            # Agent returned clarification questions (not a JSON brief)
            print("\n🤖 Agent needs more information:")
            print(brief[:500] if len(brief) > 500 else brief)
    else:
        print("\n🤖 Agent response:")
        print(final_response[:500] if final_response else "No response")
        
except Exception as e:
    import traceback
    print(f"\n❌ Error: {e}")
    traceback.print_exc()
    try:
        await close_toolsets(mcp_tools)
    except:
        pass

In [ ]:
# Check the generated brief
import json

brief = mdzen_state.get("simulation_brief")
print(f"Brief type: {type(brief)}")
print(f"Brief value:\n{brief}")

# Parse if it's a string
if isinstance(brief, str):
    try:
        brief = json.loads(brief)
        mdzen_state["simulation_brief"] = brief
        print("\n✓ Parsed successfully!")
        print("\n📋 SimulationBrief:")
        for key, val in brief.items():
            if val is not None:
                print(f"   • {key}: {val}")
    except json.JSONDecodeError as e:
        print(f"JSON parse error: {e}")


In [ ]:
# Test clarification agent with Streamable HTTP
import json

user_request = mdzen_state["user_request"]
session_dir = mdzen_state["session_dir"]

print(f"🤖 Testing clarification agent with Streamable HTTP...")
print(f"   Request: {user_request[:60]}...")
print("-" * 60)

from mdzen.agents.clarification_agent import create_clarification_agent
from mdzen.tools.mcp_setup import close_toolsets

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Create agent with HTTP transport (now uses StreamableHTTPConnectionParams)
agent, mcp_tools = create_clarification_agent(transport="http")
print("✓ Agent created with StreamableHTTPConnectionParams")

# Create runner
session_service = InMemorySessionService()
runner = Runner(
    app_name="mdzen",
    agent=agent,
    session_service=session_service,
)

async def run_clarification():
    session = await session_service.create_session(
        app_name="mdzen",
        user_id="colab_user",
        state={"session_dir": session_dir},
    )
    
    message = types.Content(
        role="user",
        parts=[types.Part(text=user_request)],
    )
    
    print("🔄 Running agent...")
    final_response = None
    async for event in runner.run_async(
        user_id="colab_user",
        session_id=session.id,
        new_message=message,
    ):
        if event.is_final_response() and event.content:
            final_response = event.content.parts[0].text if event.content.parts else None
    
    updated_session = await session_service.get_session(
        app_name="mdzen",
        user_id="colab_user",
        session_id=session.id,
    )
    
    return final_response, updated_session.state

try:
    final_response, session_state = await run_clarification()
    await close_toolsets(mcp_tools)
    
    if session_state.get("simulation_brief"):
        brief = session_state["simulation_brief"]
        if isinstance(brief, str):
            try:
                brief = json.loads(brief)
            except:
                pass
        
        if isinstance(brief, dict):
            mdzen_state["simulation_brief"] = brief
            
            print("\n✅ SUCCESS! Clarification complete!")
            print("-" * 60)
            print("📋 Generated SimulationBrief:")
            for key, val in brief.items():
                if val is not None:
                    print(f"   • {key}: {val}")
        else:
            # Agent returned clarification questions (not a JSON brief)
            print("\n🤖 Agent needs more information:")
            print(brief[:500] if len(brief) > 500 else brief)
    else:
        print("\n🤖 Agent response:")
        print(final_response[:500] if final_response else "No response")
        
except Exception as e:
    import traceback
    print(f"\n❌ Error: {e}")
    traceback.print_exc()
    try:
        await close_toolsets(mcp_tools)
    except:
        pass